# 데이터분석 기초스터디
# Chapter 10 : apply 메서드 활용

10-1 간단한 함수 만들기

In [4]:
def my_sq(x):
    return x**2

In [6]:
def my_exp(x, n):
    return x**n

In [12]:
print(my_sq(4))
print(my_exp(2,4))

16
16


10-2 apply 메서드 사용하기 - 기초

In [15]:
import pandas as pd

In [17]:
df = pd.DataFrame({'a':[10,20,30], 'b':[20,30,40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [19]:
print(df['a']**2)

0    100
1    400
2    900
Name: a, dtype: int64


In [21]:
sq = df['a'].apply(my_sq)
print(sq)

0    100
1    400
2    900
Name: a, dtype: int64


In [25]:
ex = df['a'].apply(my_exp,n=2)
print(ex)
ex1 = df['a'].apply(my_exp,n=3)
print(ex1)

0    100
1    400
2    900
Name: a, dtype: int64
0     1000
1     8000
2    27000
Name: a, dtype: int64


In [27]:
def print_me(x):
    print(x)

In [29]:
print(df.apply(print_me, axis=0))

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    None
b    None
dtype: object


In [31]:
print(df['a'])
print(df['b'])

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64


In [37]:
def avg_3(x,y,z):
    return (x+y+z)/3
# 이 함수는 데이터 프레임에 적용하면 오류가 난다. 인자가 3개 이어야 하는데 한 개만 들어가기 때문

In [39]:
def avg_3_apply(col):
    x=col[0]
    y=col[1]
    z=col[2]
    return (x+y+z)/3

In [41]:
print(df.apply(avg_3_apply))

a    20.0
b    30.0
dtype: float64


In [43]:
def avg_2_apply(row):
    sum=0
    for item in row:
        sum += item
    return sum/df.shape[1]

In [45]:
print(df.apply(avg_2_apply, axis=1))

0    15.0
1    25.0
2    35.0
dtype: float64


10-3 apply 메서드 사용하기 - 고급

In [48]:
import seaborn as sns

In [50]:
titanic = sns.load_dataset("titanic")

In [52]:
print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None


In [54]:
import numpy as np

In [58]:
# 누락값 개수를 사는 함수
def count_missing(vec):
    null_vec = pd.isnull(vec)
    null_count = np.sum(null_vec)
    return null_count

In [60]:
cmis_col = titanic.apply(count_missing)
print(cmis_col)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [64]:
# 누락값 비율
def prop_missing(vec):
    num = count_missing(vec)
    dem = vec.size
    return num/dem

In [66]:
pmis_col = titanic.apply(prop_missing)
print(pmis_col)

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64


In [68]:
def prop_complete(vec):
    return 1-prop_missing(vec)

In [70]:
cmis_row = titanic.apply(count_missing, axis=1)
pmis_row = titanic.apply(prop_missing, axis=1)
pcom_row = titanic.apply(prop_complete, axis=1)
print(cmis_row.head())

0    1
1    0
2    1
3    0
4    1
dtype: int64


In [72]:
titanic['num_missing'] = titanic.apply(count_missing, axis=1)
print(titanic.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  num_missing  
0    man        True  NaN  Southampton    no  False            1  
1  woman       False    C    Cherbourg   yes  False            0  
2  woman       False  NaN  Southampton   yes   True            1  
3  woman       False    C  Southampton   yes  False            0  
4    man        True  NaN  Southampton    no   True            1  


In [76]:
# 누락값이 2개 이상인 데이터 추출
print(titanic.loc[titanic.num_missing > 1, :].sample(10))

     survived  pclass     sex  age  sibsp  parch     fare embarked   class  \
420         0       3    male  NaN      0      0   7.8958        C   Third   
223         0       3    male  NaN      0      0   7.8958        S   Third   
552         0       3    male  NaN      0      0   7.8292        Q   Third   
359         1       3  female  NaN      0      0   7.8792        Q   Third   
121         0       3    male  NaN      0      0   8.0500        S   Third   
415         0       3  female  NaN      0      0   8.0500        S   Third   
790         0       3    male  NaN      0      0   7.7500        Q   Third   
46          0       3    male  NaN      1      0  15.5000        Q   Third   
277         0       2    male  NaN      0      0   0.0000        S  Second   
196         0       3    male  NaN      0      0   7.7500        Q   Third   

       who  adult_male deck  embark_town alive  alone  num_missing  
420    man        True  NaN    Cherbourg    no   True            2  
223